In [1]:
# !pip install gensim
# !pip install python-Levenshtein

In [2]:
# https://radimrehurek.com/gensim/
import os
import gensim
import pandas as pd
from google.colab import drive

In [3]:
drive.mount('/content/gdrive/', force_remount=True)
gdrive_path = '/content/gdrive' + '/My Drive/'
os.chdir(gdrive_path)
os.getcwd()

Mounted at /content/gdrive/


'/content/gdrive/My Drive'

In [4]:
df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [5]:
df.shape

(194439, 9)

In [6]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [7]:
# pre-process text
gensim.utils.simple_preprocess("They look good and stick good!")

['they', 'look', 'good', 'and', 'stick', 'good']

In [8]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [9]:
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [10]:
# gensim is an NLP library and comes with word2vec class
# window = 10, means 10 words before the target word and 10 after
# min_count = 2, the sentance should be at least two words to be used
# workers are the numbers of cpus
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [11]:
model.build_vocab(review_text, progress_per=1000)

In [12]:
model.epochs

5

In [13]:
model.corpus_count

194439

In [14]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61503441, 83868975)

In [15]:
model.save("word2vec-amazon-cell-reviews.model")

In [17]:
model.wv.most_similar("bad")

[('terrible', 0.680539608001709),
 ('shabby', 0.6421434283256531),
 ('horrible', 0.6017223596572876),
 ('good', 0.5962475538253784),
 ('awful', 0.5370529294013977),
 ('okay', 0.5349499583244324),
 ('cheap', 0.5286258459091187),
 ('ok', 0.5197794437408447),
 ('keen', 0.5179442763328552),
 ('poor', 0.5176799297332764)]

In [21]:
model.wv.similarity(w1="expensive", w2="expensive")

0.99999994

In [22]:
model.wv.similarity(w1="cheap", w2="expensive")

0.45277953

In [23]:
model.wv.similarity(w1="cheap", w2="inexpensive")

0.5217233

In [24]:
model.wv.similarity(w1="good", w2="great")

0.77414644